In [3]:
import polars as pl
import duckdb
import plotly.express as px
import sys
import logging
from pathlib import Path
from IPython.display import display
import time

# --- Configuration ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# --- Helper Function ---
def create_visualization(df):
    """Creates and returns a Plotly line chart."""
    try:
        pandas_df = df.to_pandas()
        fig = px.line(
            pandas_df,
            x='date',
            y='close',
            color='symbol',
            title='Close Prices Comparison: Reliance vs. TCS',
            labels={'date': 'Date', 'close': 'Closing Price (INR)', 'symbol': 'Stock Symbol'}
        )
        fig.update_layout(legend_title='Symbol', hovermode='x unified')
        fig.update_xaxes(rangeslider_visible=True)
        return fig
    except Exception as e:
        logger.error(f"Error creating visualization: {e}")
        return None

# --- Enhanced Connection Management ---
def get_fresh_connection():
    """Get a fresh DuckDB connection with proper error handling"""
    try:
        # First try to use the config connection
        project_root = Path.cwd().parent
        if str(project_root) not in sys.path:
            sys.path.append(str(project_root))
        
        from quandex_core.config import config
        
        # Force a fresh connection by reconnecting
        db_path = config.data.duckdb_path
        logger.info(f"Connecting to database at: {db_path}")
        
        # Create a new connection (don't use the shared one)
        fresh_conn = duckdb.connect(str(db_path))
        
        # Test the connection
        fresh_conn.execute("SELECT 1").fetchone()
        logger.info("Fresh connection established successfully")
        
        return fresh_conn, db_path
        
    except Exception as e:
        logger.error(f"Error establishing fresh connection: {e}")
        
        # Fallback: try direct connection
        try:
            fallback_path = Path.cwd().parent / "data_vault" / "market_boards" / "quandex.duckdb"
            logger.info(f"Trying fallback connection to: {fallback_path}")
            fresh_conn = duckdb.connect(str(fallback_path))
            fresh_conn.execute("SELECT 1").fetchone()
            logger.info("Fallback connection established successfully")
            return fresh_conn, fallback_path
        except Exception as fallback_error:
            logger.error(f"Fallback connection also failed: {fallback_error}")
            raise

def diagnose_database(conn):
    """Diagnose database state"""
    try:
        # List all tables
        tables = conn.execute("SHOW TABLES").fetchall()
        logger.info(f"Available tables: {[t[0] for t in tables]}")
        
        # Check if processed_equity_data exists
        table_exists = any(t[0] == 'processed_equity_data' for t in tables)
        
        if table_exists:
            count = conn.execute("SELECT COUNT(*) FROM processed_equity_data").fetchone()[0]
            logger.info(f"processed_equity_data table has {count} records")
            
            # Check unique symbols
            symbols = conn.execute("SELECT DISTINCT symbol FROM processed_equity_data ORDER BY symbol").fetchall()
            logger.info(f"Available symbols: {[s[0] for s in symbols]}")
            
            return True, count
        else:
            logger.warning("processed_equity_data table does not exist!")
            return False, 0
            
    except Exception as e:
        logger.error(f"Error during database diagnosis: {e}")
        return False, 0

# --- Main Analysis Block ---
def main_analysis():
    """Main analysis function with robust connection handling"""
    conn = None
    try:
        # Get a fresh connection
        conn, db_path = get_fresh_connection()
        
        # Diagnose database state
        table_exists, record_count = diagnose_database(conn)
        
        if not table_exists:
            logger.error("processed_equity_data table doesn't exist!")
            logger.info("Please run the data fetcher first:")
            logger.info("python -m quandex_core.data_engine.nse_equity_fetcher")
            return
            
        if record_count == 0:
            logger.warning("processed_equity_data table is empty!")
            return
        
        # --- Database Query with Error Handling ---
        query = """
        SELECT
            symbol,
            date,
            close,
            sma_50,
            rsi_14
        FROM processed_equity_data
        WHERE symbol IN (?, ?)
        ORDER BY date
        """
        
        logger.info("Executing query on processed_equity_data table...")
        
        # Execute query with parameters
        results_df = conn.execute(query, ['RELIANCE.NS', 'TCS.NS']).pl()

        if results_df.is_empty():
            logger.warning("No data returned for RELIANCE.NS and TCS.NS!")
            
            # Check what symbols are actually available
            available_symbols = conn.execute("SELECT DISTINCT symbol FROM processed_equity_data").fetchall()
            logger.info(f"Available symbols in database: {[s[0] for s in available_symbols]}")
            
            # Try with available symbols if the requested ones don't exist
            if available_symbols:
                first_two_symbols = [s[0] for s in available_symbols[:2]]
                logger.info(f"Trying with available symbols: {first_two_symbols}")
                results_df = conn.execute(query, first_two_symbols).pl()
        
        if results_df.is_empty():
            logger.error("Still no data returned!")
            return

        logger.info(f"Retrieved {len(results_df)} records successfully!")
        
        # Display data preview
        display(results_df.head(10))
        
        # Show data summary
        logger.info("Data summary:")
        summary = results_df.group_by("symbol").agg([
            pl.col("close").count().alias("record_count"),
            pl.col("date").min().alias("start_date"),
            pl.col("date").max().alias("end_date"),
            pl.col("close").mean().alias("avg_close")
        ])
        display(summary)

        # --- Visualization ---
        logger.info("Creating visualization...")
        fig = create_visualization(results_df)
        if fig:
            fig.show()
        else:
            logger.error("Failed to create visualization")
            
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
        
    finally:
        if conn:
            try:
                conn.close()
                logger.info("Database connection closed")
            except:
                pass
        logger.info("Analysis completed")

# --- Run the analysis ---
if __name__ == "__main__":
    main_analysis()

2025-06-14 00:03:14 | INFO     | Connecting to database at: data_vault\market_boards\quandex.duckdb
2025-06-14 00:03:14 | INFO     | Fresh connection established successfully
2025-06-14 00:03:14 | INFO     | Available tables: []
2025-06-14 00:03:14 | WARNING  | processed_equity_data table does not exist!
2025-06-14 00:03:14 | ERROR    | processed_equity_data table doesn't exist!
2025-06-14 00:03:14 | INFO     | Please run the data fetcher first:
2025-06-14 00:03:14 | INFO     | python -m quandex_core.data_engine.nse_equity_fetcher
2025-06-14 00:03:14 | INFO     | Database connection closed
2025-06-14 00:03:14 | INFO     | Analysis completed
